In [ ]:
!pip install nltk numpy openpyxl Pillow pyparsing scikit-learn scipy keras scipy tensorflow transformers tensorflow-text tf-models-official

     |████████████████████████████████| 2.1MB 11.3MB/s 
     |████████████████████████████████| 3.4MB 38.3MB/s 
     |████████████████████████████████| 1.1MB 30.5MB/s 
     |████████████████████████████████| 901kB 31.8MB/s 
     |████████████████████████████████| 3.3MB 34.8MB/s 
     |████████████████████████████████| 38.2MB 72kB/s 
     |████████████████████████████████| 174kB 43.9MB/s 
     |████████████████████████████████| 706kB 31.6MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 358kB 36.5MB/s 
     |████████████████████████████████| 1.2MB 32.8MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22245 sha256=660d8438d832c122c8e7adb91c0626e73103d0d87ae13f2d8dd61424ebb04cc7
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any

In [ ]:
import nltk
import re
from openpyxl import load_workbook
from sklearn import preprocessing
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from transformers import BertTokenizer, TFBertForSequenceClassification
import os
import shutil
import matplotlib.pyplot as plt
tf.get_logger().setLevel("ERROR")

In [ ]:
ALL_CLASSES = [
    "content discussion", "greeting", "logistics", "assignment instructions", "instruction question",
    "assignment question", "general comment", "response", "incomplete/typo", "feedback",
    "emoticon/non-verbal", "discussion wrap-up", "outside material", "opening statement",
    "general question", "content question", "general discussion"
]
DATASET_PATH = "dataset.xlsx"

# Opens a single sheet for reading, returns a list of messages and a list of classes
def read_worksheet(filename, sheet_name, all_classes, label_encoder, no_columns):
    wb = load_workbook(filename, read_only=True)
    ws = wb[sheet_name]
    column_labels = next(ws.rows)
    X = []
    y = []
    for row in ws.rows:
        if row[0].value is None:
            break
        elif row[0].value.strip() == "Course":  # Skip the first line which only contains column titles
            continue
        new_entry = {}
        for i in range(no_columns):
            new_entry[column_labels[i].value.lower().replace(" ", "_")] = str(row[i].value)
        c_list = [new_entry["codepreliminary"].lower().strip()]
        if c_list[0] not in all_classes:
            c_list = new_entry["codepreliminary"].lower().strip().split("/")
        # If there are 2 classes listed in document add message twice (1 for each class)
        for c in c_list:
            new_entry["codepreliminary"] = label_encoder.transform([c])[0]
            X.append(new_entry["message"])
            y.append(label_encoder.transform([c])[0])
    wb.close()
    return X, y

le = preprocessing.LabelEncoder()
le.fit(ALL_CLASSES)

LabelEncoder()

In [ ]:
X, y = read_worksheet(DATASET_PATH, "Discussion only data", ALL_CLASSES, le, 10)
X2, y2 = read_worksheet(DATASET_PATH, "CREW data", ALL_CLASSES, le, 11)
X.extend(X2)  # Join worksheets into a single dataset
y.extend(y2)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
max_length = 256
batch_size = 6

In [ ]:
def map_data_to_dict(input_ids, attention_masks, token_type_ids, classes):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, classes

In [ ]:
def encode_data(data, classes):
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  class_list = []
  for i, msg in enumerate(data):
    input = tokenizer.encode_plus(
              msg,
              add_special_tokens=True,
              max_length=max_length,
              pad_to_max_length=True,
              return_attention_mask=True
            )
    input_ids_list.append(input["input_ids"])
    token_type_ids_list.append(input["token_type_ids"])
    attention_mask_list.append(input["attention_mask"])
    class_list.append([classes[i]])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, class_list)).map(map_data_to_dict)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = encode_data(X_train, y_train).shuffle(5000).batch(batch_size)
test_dataset = encode_data(X_test, y_test).batch(batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
def build_model(learning_rate=3e-5, no_epochs=5):
  model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(ALL_CLASSES))
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

  model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
  return model

In [ ]:
def get_results(test_data, model, print_output=True):
  results_true = test_data.unbatch()
  results_true = np.asarray([element[1].numpy() for element in results_true])
  results = model.predict(test_data)
  results_predicted = np.argmax(results.logits, axis=1)

  accuracy = accuracy_score(results_true, results_predicted)
  precision = precision_score(results_true, results_predicted, zero_division=1, average="weighted")
  recall = recall_score(results_true, results_predicted, zero_division=1, average="weighted")
  f1 = f1_score(results_true, results_predicted, zero_division=1, average="weighted")

  if print_output:
    print("Accuracy score: {}".format(accuracy))
    print("Precision score: {}".format(precision))
    print("Recall score: {}".format(recall))
    print("F1 score: {}".format(f1))

  return accuracy, precision, recall, f1

In [ ]:
model = build_model()
history = model.fit(train_dataset, epochs=no_epochs, validation_data=test_dataset)
get_results(test_dataset, model)

Epoch 1/5
113/113 [==============================] - 56s 370ms/step - loss: 2.2922 - accuracy: 0.3722 - val_loss: 1.5965 - val_accuracy: 0.5325
Epoch 2/5
113/113 [==============================] - 40s 353ms/step - loss: 1.3349 - accuracy: 0.6194 - val_loss: 1.3641 - val_accuracy: 0.6272
Epoch 3/5
113/113 [==============================] - 40s 356ms/step - loss: 0.9220 - accuracy: 0.7424 - val_loss: 1.2910 - val_accuracy: 0.6450
Epoch 4/5
113/113 [==============================] - 41s 359ms/step - loss: 0.5909 - accuracy: 0.8678 - val_loss: 1.3272 - val_accuracy: 0.6627
Epoch 5/5
113/113 [==============================] - 41s 360ms/step - loss: 0.3867 - accuracy: 0.9264 - val_loss: 1.2713 - val_accuracy: 0.6272


In [ ]:
no_folds = 5
no_epochs = 5
kfold = KFold(no_folds, shuffle=True)

# Convert to numpy arrays so indexing with list of indexes works
X = np.array(X) 
y = np.array(y)

test_accuracy_log = []

accuracy = []
precision = []
recall = []
f1 = []
for train, test in kfold.split(X, y):
  X_train = X[train]
  y_train = y[train]
  X_test = X[test]
  y_test = y[test]

  train_dataset = encode_data(X_train, y_train).shuffle(5000).batch(batch_size)
  test_dataset = encode_data(X_test, y_test).batch(batch_size)
  
  model = build_model()
  model.fit(train_dataset, epochs=no_epochs, validation_data=test_dataset, verbose=1)

  acc, pre, rec, f = get_results(test_dataset, model, print_output=False)
  accuracy.append(acc)
  precision.append(pre)
  recall.append(rec)
  f1.append(f)

print("Accuracy score: {}".format(np.mean(accuracy)))
print("Precision score: {}".format(np.mean(precision)))
print("Recall score: {}".format(np.mean(recall)))
print("F1 score: {}".format(np.mean(f1)))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 97s 701ms/step - loss: 2.1424 - accuracy: 0.4571 - val_loss: 1.4242 - val_accuracy: 0.6095
Epoch 2/5
113/113 [==============================] - 76s 677ms/step - loss: 1.2918 - accuracy: 0.6570 - val_loss: 1.1665 - val_accuracy: 0.6627
Epoch 3/5
113/113 [==============================] - 76s 677ms/step - loss: 0.9670 - accuracy: 0.7276 - val_loss: 1.0456 - val_accuracy: 0.7160
Epoch 4/5
113/113 [==============================] - 76s 677ms/step - loss: 0.5852 - accuracy: 0.8517 - val_loss: 1.0065 - val_accuracy: 0.7337
Epoch 5/5
113/113 [==============================] - 76s 677ms/step - loss: 0.4278 - accuracy: 0.9058 - val_loss: 0.9873 - val_accuracy: 0.7278


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 98s 701ms/step - loss: 2.1805 - accuracy: 0.3929 - val_loss: 1.3552 - val_accuracy: 0.6272
Epoch 2/5
113/113 [==============================] - 76s 676ms/step - loss: 1.2566 - accuracy: 0.6779 - val_loss: 1.2165 - val_accuracy: 0.6272
Epoch 3/5
113/113 [==============================] - 76s 676ms/step - loss: 0.7903 - accuracy: 0.7997 - val_loss: 1.0944 - val_accuracy: 0.6568
Epoch 4/5
113/113 [==============================] - 76s 676ms/step - loss: 0.5174 - accuracy: 0.8823 - val_loss: 1.0666 - val_accuracy: 0.6627
Epoch 5/5
113/113 [==============================] - 76s 676ms/step - loss: 0.2770 - accuracy: 0.9449 - val_loss: 1.1454 - val_accuracy: 0.7101


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 98s 706ms/step - loss: 2.1839 - accuracy: 0.4207 - val_loss: 1.4850 - val_accuracy: 0.6331
Epoch 2/5
113/113 [==============================] - 76s 676ms/step - loss: 1.2083 - accuracy: 0.6898 - val_loss: 1.3739 - val_accuracy: 0.6509
Epoch 3/5
113/113 [==============================] - 76s 676ms/step - loss: 0.8214 - accuracy: 0.7710 - val_loss: 1.2052 - val_accuracy: 0.6627
Epoch 4/5
113/113 [==============================] - 76s 676ms/step - loss: 0.6125 - accuracy: 0.8633 - val_loss: 1.0358 - val_accuracy: 0.7278
Epoch 5/5
113/113 [==============================] - 76s 676ms/step - loss: 0.3956 - accuracy: 0.9177 - val_loss: 1.1751 - val_accuracy: 0.7219


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 98s 703ms/step - loss: 2.2601 - accuracy: 0.3680 - val_loss: 1.4011 - val_accuracy: 0.6310
Epoch 2/5
113/113 [==============================] - 76s 677ms/step - loss: 1.1876 - accuracy: 0.6659 - val_loss: 1.2030 - val_accuracy: 0.6488
Epoch 3/5
113/113 [==============================] - 76s 676ms/step - loss: 0.8074 - accuracy: 0.7700 - val_loss: 1.1203 - val_accuracy: 0.7143
Epoch 4/5
113/113 [==============================] - 76s 676ms/step - loss: 0.5615 - accuracy: 0.8719 - val_loss: 1.1313 - val_accuracy: 0.7024
Epoch 5/5
113/113 [==============================] - 76s 676ms/step - loss: 0.3903 - accuracy: 0.9257 - val_loss: 1.1702 - val_accuracy: 0.7083


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 98s 701ms/step - loss: 2.3692 - accuracy: 0.3105 - val_loss: 1.6254 - val_accuracy: 0.5417
Epoch 2/5
113/113 [==============================] - 76s 676ms/step - loss: 1.3777 - accuracy: 0.6170 - val_loss: 1.2267 - val_accuracy: 0.6131
Epoch 3/5
113/113 [==============================] - 76s 677ms/step - loss: 1.0143 - accuracy: 0.7012 - val_loss: 1.0758 - val_accuracy: 0.6726
Epoch 4/5
113/113 [==============================] - 76s 676ms/step - loss: 0.7786 - accuracy: 0.7624 - val_loss: 0.9452 - val_accuracy: 0.7262
Epoch 5/5
113/113 [==============================] - 76s 675ms/step - loss: 0.5473 - accuracy: 0.8517 - val_loss: 0.9368 - val_accuracy: 0.7500
Accuracy score: 0.7236193293885602
Precision score: 0.7652244631480469
Recall score: 0.7236193293885602
F1 score: 0.7058959170966468


In [ ]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training')
    plt.plot(x, val_acc, 'r', label='Validation')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training')
    plt.plot(x, val_loss, 'r', label='Validation')
    plt.title('Training and validation loss')
    plt.legend()
  
plot_history(history)